In [1]:
import firebase_admin
from firebase_admin import credentials, firestore

# cred = credentials.Certificate("../recyclesg-firebase-adminsdk-serviceAccountKey.json")
# cred = credentials.Certificate("../serviceAccountKey.json")

# app = firebase_admin.initialize_app(
#     credential=cred,
# )
app = firebase_admin.initialize_app()

db = firestore.client()



In [1]:
from google.cloud import firestore
client = firestore.AsyncClient()
collection = client.collection('test')

In [55]:
from google.cloud.firestore_v1.base_query import FieldFilter

res = collection.where(filter=FieldFilter("id", "==", "3")).where(filter=FieldFilter("address.city", "==", "Anytown"))
res

In [56]:
docs = await res.limit(1).get()
docs

In [41]:
doc_ref = docs[0].reference
doc_ref

In [43]:
await doc_ref.set({"address": {"street": "125 Main St", "city": "Anytown", "zip": "12345"}}, merge=True)

update_time {
  seconds: 1723365166
  nanos: 517744000
}

#### Batched writes

In [44]:
batch = client.batch()
batch

In [45]:
add_res = await collection.add({"id": 4, "name": "zhiwei"})
add_res

(DatetimeWithNanoseconds(2024, 8, 11, 8, 49, 43, 372752, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.async_document.AsyncDocumentReference at 0x7f42e43825c0>)

In [53]:
doc_ref = add_res[1]
await doc_ref.set({"address": {"street": "125 Main St", "city": "Anytown", "zip": "12345"}}, merge=True)

update_time {
  seconds: 1723366249
  nanos: 683649000
}

In [ ]:
batch.set()

In [35]:
async for doc in res.stream():
    doc = doc.to_dict()
    print(f"{doc.get('id', None)} => {doc.get('name', None)}")


3 => John
3 => Johnny


In [8]:
client.close()

In [29]:
await collection.document("1").set({"address": {"street": "124 Main St", "city": "Anytown", "zip": "12345"}}, merge=True)

update_time {
  seconds: 1723363631
  nanos: 239801000
}

In [54]:
doc = await collection.document("2").get()
doc


# Showing all collections in the db

In [3]:
for coll in db.collections():
    print(coll.id)

HistoryMessages
nea-items
recyclable
recyclerequests
search
test
typesense_sync
user
users
view_search_results
wasteType
wasteTypeFeedback


In [13]:
collection = db.collection('wasteType')

In [12]:
collection.document("2").get().to_dict()

{'messages': ['human', "what's the weather in singapore"]}

I0000 00:00:1722780349.513261  123599 tcp_posix.cc:809] IOMGR endpoint shutdown


In [11]:
data = {"messages": ["human", "what's the weather in singapore"]}
collection.document("2").set(data)

update_time {
  seconds: 1722780101
  nanos: 770815000
}

# Getting documents in `wasteType` collection

In [3]:
collection = db.collection("wasteType")

In [4]:
# Get all docs in a collection
docs = collection.get()
print(f"There are {len(docs)} documents in the collection")

docs[0].to_dict()

There are 430 documents in the collection


{'links': ['https://www.nea.gov.sg/our-services/waste-management/3r-programmes-and-resources/e-waste-management/where-to-recycle-e-waste'],
 'instructions': 'Can be recycled through: • Retailer 1-for-1 Take-back• Bulky item removal service by TCs for HDB residents• E-waste collection drive (Quarterly)• Doorstep collection (fees may apply) To find out where you can recycle your e-waste, visit here.*Please note that the small household appliances will be collected and recycled under recycling programmes organised under the National Voluntary Partnership for the Proper Management of Non-Regulated Used Household Electrical/Electronic Products.',
 'item': 'nintendo',
 'recyclable': True,
 'material': 'Others'}

In [14]:
# Get all docs in a collection
docs = db.collection("wasteType").stream()

items_db = []
for doc in docs:
    items_db.append(doc.to_dict())
len(items_db)



436

## Deleting documents from a collection

In [ ]:
# Function to delete a collection
def delete_collection(coll_ref, batch_size):
    docs = coll_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        print(f'Deleting doc {doc.id} => {doc.to_dict()}')
        doc.reference.delete()
        deleted += 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

# Usage
delete_collection(collection, batch_size=50)



# Querying a collection

In [5]:
# get a document in a collection by item name
# Note: Use of CollectionRef stream() is prefered to get()
from google.cloud.firestore_v1.base_query import FieldFilter

docs = (
    collection
    .where(filter=FieldFilter("item", "==", "desktop computer"))
    .stream()
)

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

0VQD4HtaoNsBaZoCsJsk => {'links': ['https://www.nea.gov.sg/our-services/waste-management/3r-programmes-and-resources/e-waste-management/where-to-recycle-e-waste'], 'instructions': 'Can be recycled through E-waste bins, located at the link(s).', 'item': 'desktop computer', 'recyclable': True, 'material': 'Others'}


In [3]:
# find by id
db.collection("wasteType").document("CNUeuF9SfghzfAMxGC1Z").get().to_dict()

{'links': [],
 'instructions': 'Should be disposed of as general waste.',
 'item': 'debit card',
 'recyclable': False,
 'material': 'Plastic'}

I0000 00:00:1722233070.900497  100584 tcp_posix.cc:809] IOMGR endpoint shutdown


In [7]:
# text = "Can be recycled through: \u2022 Retailer 1-for-1 Take-back\u2022 Bulky item removal service by TCs for HDB residents\u2022 E-waste collection drive (Quarterly)\u2022 Doorstep collection (fees may apply) To find out where you can recycle your e-waste, visit here.*Please note that the small household appliances will be collected and recycled under recycling programmes organised under the National Voluntary Partnership for the Proper Management of Non-Regulated Used Household Electrical/Electronic Products."
text = "make sure it is clean before recycling"
text.split('\u2022')

['make sure it is clean before recycling']

# Seeding data into database

In [18]:
import json
# Start a batch
batch = db.batch()

# Data to add
with open('../data/cleaned_data_final.json', 'r') as file:
    data = json.load(file)

# Add documents to batch
for doc_data in data:
    doc_data['item'] = doc_data['item'].lower()
    doc_ref = collection.document()  # Create a new document reference
    batch.set(doc_ref, doc_data)  # Add to batch

# Commit the batch
batch.commit()

[update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_time {
   seconds: 1719459499
   nanos: 572925000
 },
 update_

In [20]:
# Note: Use of CollectionRef stream() is prefered to get()
docs = (
    collection
    .where(filter=FieldFilter("item", "in", ["eraser", "vegetables", "pencil box"]))
    .stream()
)

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

0e1kC4AfnFSkeTyUR8i8 => {'instructions': 'should be disposed of as general waste.', 'item': 'vegetables', 'recyclable': False, 'material': 'Others', 'id': '47850322-8c61-4f32-ab37-8bb1a24a98fd'}
AUTVwYEMliQuyeRP2yjY => {'instructions': 'Donate if it is in good condition.', 'item': 'eraser', 'recyclable': False, 'material': 'Others', 'id': '0937acbe-5571-4919-9d9c-897cd236da02'}


In [51]:
collection = db.collection("test")

In [34]:
from google.cloud.firestore_v1.base_query import FieldFilter
from datetime import datetime

# Note: Use of CollectionRef stream() is prefered to get()
check_recyclable = True
check_material = True

# query = (
#     collection
#     .where(filter=FieldFilter("recyclable", "==", True))
    # .where(filter=FieldFilter("material", "==", "Paper"))
    # .order_by("createdAt", direction="ASCENDING")
    # .limit(1)
# )
query = collection
if check_recyclable:
    query = query.where(filter=FieldFilter("recyclable", "==", True))
if check_material:
    query = query.where(filter=FieldFilter("material", "==", "Paper"))
query = query.where(filter=FieldFilter("createdAt", "<", datetime.now()))

docs = query.order_by("createdAt", direction="ASCENDING").limit(2).stream()

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")

jzGaGQCvJxZpGnlBUQBr => {'createdAt': '2024-04-29T12:16:42.482Z', 'tags': [], 'description': 'Make sure it is clean before recycling.', 'recyclable': True, 'material': 'Paper', 'name': 'Writing Paper'}
FQoI61pfq3OZU0lX1RBi => {'createdAt': '2024-04-29T12:17:19.801Z', 'tags': [], 'description': 'Make sure it is clean before recycling.', 'recyclable': True, 'material': 'Paper', 'name': 'Paper'}


I0000 00:00:1722786499.516626  123599 tcp_posix.cc:809] IOMGR endpoint shutdown


In [ ]:
docs = collection.order_by("content").limit(2)

for doc in docs.stream():
    print(f"{doc.id} => {doc.to_dict()}")


I0000 00:00:1722789419.516442  123599 tcp_posix.cc:809] IOMGR endpoint shutdown


In [27]:
collection.document("0CyfrRmlfXAoeIUwxICX").get().to_dict()

{'links': [],
 'instructions': 'Make sure it is clean before recycling.',
 'item': 'gift bag',
 'recyclable': True,
 'material': 'Paper'}

I0000 00:00:1722783114.515756  123536 tcp_posix.cc:809] IOMGR endpoint shutdown
